In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']
        if 'static_grid' not in config_type:
            continue

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4: key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_868540/3078009364.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_868540/3078009364.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_868540/3078009364.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_868540/3078009364.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\1000XECUSDT\plots\2022-03-15T184506\backtest_...,5.5,1000XECUSDT,True,13.140,0.06520,100.0,0.9340,True,29.910,0.1540,100.0,0.9526
1,\1000XECUSDT\plots\2022-03-16T132802\backtest_...,5.5,1000XECUSDT,True,13.410,0.06740,100.0,0.9340,True,30.410,0.1440,100.0,0.9526
2,\1INCHUSDT\plots\2022-03-16T133853\backtest_re...,5.5,1INCHUSDT,True,2.589,0.00706,100.0,0.8369,True,26.760,0.0778,100.0,0.7473
3,\AAVEUSDT\plots\2022-03-14T073611\backtest_res...,5.5,AAVEUSDT,True,-11.590,-0.05940,100.0,0.7649,True,54.850,0.2140,100.0,0.7848
4,\ADAUSDT\plots\2022-03-16T140203\backtest_resu...,5.5,ADAUSDT,True,15.100,0.04030,100.0,0.9334,True,9.646,0.0328,100.0,0.8184
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,\ZENUSDT\plots\2022-03-17T013136\backtest_resu...,5.5,ZENUSDT,True,13.990,0.03850,100.0,0.8480,True,31.140,0.0897,100.0,0.8627
126,\ZILUSDT\plots\2022-03-17T013628\backtest_resu...,5.5,ZILUSDT,True,-9.182,-0.03070,100.0,0.8849,True,17.220,0.0519,100.0,0.8273
127,\ZILUSDT\plots\2022-03-29T143724\backtest_resu...,5.5,ZILUSDT,True,3.951,0.01720,100.0,0.8952,True,9.160,-0.0188,100.0,0.8489
128,\ZILUSDT\plots\2022-03-29T145111\backtest_resu...,5.5,ZILUSDT,True,13.630,0.05290,100.0,0.9544,True,15.840,-0.0347,100.0,0.7411


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["RUNE", "WAVES", "ETC", "DGB",
                   "ZIL",
                   "HOT",
                   "OCEAN",
                   "ALPHA",
                   "ADA",
                   "SXP", ]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['adgl_%'] > 0.04)
              #& (big_data['short'] == True)
              #& (big_data['adgt_%'] > 0.1)
              & (big_data['bnkrl_%'] > 95.0)
              #& (big_data['bnkrs_%'] > 95.0)
              #& (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
4,\ADAUSDT\plots\2022-03-16T140203\backtest_resu...,5.5,ADAUSDT,True,15.10,0.0403,100.0,0.9334,True,9.646,0.0328,100.0,0.8184,0.0731
6,\ADAUSDT\plots\2022-03-29T144346\backtest_resu...,5.5,ADAUSDT,True,11.60,0.0481,100.0,0.9484,True,20.640,0.0526,100.0,0.9109,0.1007
14,\ALPHAUSDT\plots\2022-03-29T144453\backtest_re...,5.5,ALPHAUSDT,True,23.10,0.0719,100.0,0.9334,True,22.460,0.0663,100.0,0.9495,0.1382
33,\DGBUSDT\plots\2022-03-07T185158\backtest_resu...,5.5,DGBUSDT,True,11.75,0.0518,100.0,0.9583,True,22.520,0.0890,100.0,0.9309,0.1408
34,\DGBUSDT\plots\2022-03-07T211718\backtest_resu...,5.5,DGBUSDT,True,11.75,0.0518,100.0,0.9583,True,22.520,0.0890,100.0,0.9309,0.1408
37,\DGBUSDT\plots\2022-03-29T144558\backtest_resu...,5.5,DGBUSDT,True,12.31,0.0483,100.0,0.9583,True,23.370,0.0120,100.0,0.7944,0.0603
47,\ETCUSDT\plots\2022-03-28T174027\backtest_resu...,5.5,ETCUSDT,True,10.76,0.0436,100.0,0.9477,True,24.020,0.0269,100.0,0.8363,0.0705
58,\HOTUSDT\plots\2022-03-29T143822\backtest_resu...,5.5,HOTUSDT,True,11.51,0.0455,100.0,0.9216,True,11.340,0.0232,100.0,0.9127,0.0687
59,\HOTUSDT\plots\2022-03-29T144701\backtest_resu...,5.5,HOTUSDT,True,14.65,0.0566,100.0,0.9304,True,25.900,0.0399,100.0,0.8540,0.0965
81,\OCEANUSDT\plots\2022-03-29T143926\backtest_re...,5.5,OCEANUSDT,True,13.25,0.0514,100.0,0.9447,True,7.734,0.0181,100.0,0.9208,0.0695


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_868540/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
4,\ADAUSDT\plots\2022-03-16T140203\backtest_resu...,ADAUSDT,0.0731,0.0403,0.0328,100.0,100.0,0.9334,0.8184
6,\ADAUSDT\plots\2022-03-29T144346\backtest_resu...,ADAUSDT,0.1007,0.0481,0.0526,100.0,100.0,0.9484,0.9109
14,\ALPHAUSDT\plots\2022-03-29T144453\backtest_re...,ALPHAUSDT,0.1382,0.0719,0.0663,100.0,100.0,0.9334,0.9495
33,\DGBUSDT\plots\2022-03-07T185158\backtest_resu...,DGBUSDT,0.1408,0.0518,0.0890,100.0,100.0,0.9583,0.9309
37,\DGBUSDT\plots\2022-03-29T144558\backtest_resu...,DGBUSDT,0.0603,0.0483,0.0120,100.0,100.0,0.9583,0.7944
47,\ETCUSDT\plots\2022-03-28T174027\backtest_resu...,ETCUSDT,0.0705,0.0436,0.0269,100.0,100.0,0.9477,0.8363
58,\HOTUSDT\plots\2022-03-29T143822\backtest_resu...,HOTUSDT,0.0687,0.0455,0.0232,100.0,100.0,0.9216,0.9127
59,\HOTUSDT\plots\2022-03-29T144701\backtest_resu...,HOTUSDT,0.0965,0.0566,0.0399,100.0,100.0,0.9304,0.8540
81,\OCEANUSDT\plots\2022-03-29T143926\backtest_re...,OCEANUSDT,0.0695,0.0514,0.0181,100.0,100.0,0.9447,0.9208
82,\OCEANUSDT\plots\2022-03-29T144810\backtest_re...,OCEANUSDT,0.1238,0.0865,0.0373,100.0,100.0,0.9442,0.8995
